In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ds=pd.read_csv('dataset.csv')
df=pd.DataFrame(ds)
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [2]:
df.drop('Loan_ID',axis=1,inplace=True)
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [3]:
df.isnull().sum().sort_values(ascending=False)

Credit_History       50
Self_Employed        32
LoanAmount           22
Dependents           15
Loan_Amount_Term     14
Gender               13
Married               3
Education             0
ApplicantIncome       0
CoapplicantIncome     0
Property_Area         0
Loan_Status           0
dtype: int64

In [4]:
#remove missing values
null_cols = ['Credit_History', 'Self_Employed', 'LoanAmount','Dependents', 'Loan_Amount_Term', 'Gender', 'Married']

for col in null_cols:
    df[col] = df[col].fillna(
    df[col].dropna().mode().values[0] )   

df.isnull().sum().sort_values(ascending=False)

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [5]:
#convert categorical data to numeric
to_numeric = {'Male': 1, 'Female': 2,
'Yes': 1, 'No': 2,
'Graduate': 1, 'Not Graduate': 2,
'Urban': 3, 'Semiurban': 2,'Rural': 1,
'Y': 1, 'N': 0,
'3+': 3}
df = df.applymap(lambda label: to_numeric.get(label) if label in to_numeric else label)
Dependents_ = pd.to_numeric(df.Dependents)
df.drop(['Dependents'], axis = 1, inplace = True)
df = pd.concat([df, Dependents_], axis = 1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             614 non-null    int64  
 1   Married            614 non-null    int64  
 2   Education          614 non-null    int64  
 3   Self_Employed      614 non-null    int64  
 4   ApplicantIncome    614 non-null    int64  
 5   CoapplicantIncome  614 non-null    float64
 6   LoanAmount         614 non-null    float64
 7   Loan_Amount_Term   614 non-null    float64
 8   Credit_History     614 non-null    float64
 9   Property_Area      614 non-null    int64  
 10  Loan_Status        614 non-null    int64  
 11  Dependents         614 non-null    int64  
dtypes: float64(4), int64(8)
memory usage: 57.7 KB
None


In [6]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [8]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
DT.fit(X_train, y_train)
y_pred = DT.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
cr=(classification_report(y_test, y_pred))
acc = accuracy_score(y_pred,y_test)
print('Confusion matrix:')
print(cm)
print('Classification_report:')
print(cr)
print('Accuracy_score',acc)

Confusion matrix:
[[72 10 11  8]
 [11  5  4  3]
 [ 5  3  7  4]
 [ 8  2  1  0]]
Classification_report:
              precision    recall  f1-score   support

           0       0.75      0.71      0.73       101
           1       0.25      0.22      0.23        23
           2       0.30      0.37      0.33        19
           3       0.00      0.00      0.00        11

    accuracy                           0.55       154
   macro avg       0.33      0.32      0.32       154
weighted avg       0.57      0.55      0.56       154

Accuracy_score 0.5454545454545454


In [9]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier()
RF.fit(X_train, y_train)
y_pred = RF.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
cr=(classification_report(y_test, y_pred))
acc = accuracy_score(y_pred,y_test)
print('Confusion matrix:')
print(cm)
print('Classification_report:')
print(cr)
print('Accuracy_score',acc)

Confusion matrix:
[[91  5  5  0]
 [12  3  8  0]
 [13  1  4  1]
 [ 9  0  2  0]]
Classification_report:
              precision    recall  f1-score   support

           0       0.73      0.90      0.81       101
           1       0.33      0.13      0.19        23
           2       0.21      0.21      0.21        19
           3       0.00      0.00      0.00        11

    accuracy                           0.64       154
   macro avg       0.32      0.31      0.30       154
weighted avg       0.55      0.64      0.58       154

Accuracy_score 0.6363636363636364


In [10]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
cr=(classification_report(y_test, y_pred))
acc = accuracy_score(y_pred,y_test)
print('Confusion matrix:')
print(cm)
print('Classification_report:')
print(cr)
print('Accuracy_score',acc)

Confusion matrix:
[[101   0   0   0]
 [ 23   0   0   0]
 [ 19   0   0   0]
 [ 11   0   0   0]]
Classification_report:
              precision    recall  f1-score   support

           0       0.66      1.00      0.79       101
           1       0.00      0.00      0.00        23
           2       0.00      0.00      0.00        19
           3       0.00      0.00      0.00        11

    accuracy                           0.66       154
   macro avg       0.16      0.25      0.20       154
weighted avg       0.43      0.66      0.52       154

Accuracy_score 0.6558441558441559


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train, y_train)
y_pred = LR.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
cr=(classification_report(y_test, y_pred))
acc = accuracy_score(y_pred,y_test)
print('Confusion matrix:')
print(cm)
print('Classification_report:')
print(cr)
print('Accuracy_score',acc)

Confusion matrix:
[[98  2  0  1]
 [21  1  0  1]
 [19  0  0  0]
 [11  0  0  0]]
Classification_report:
              precision    recall  f1-score   support

           0       0.66      0.97      0.78       101
           1       0.33      0.04      0.08        23
           2       0.00      0.00      0.00        19
           3       0.00      0.00      0.00        11

    accuracy                           0.64       154
   macro avg       0.25      0.25      0.22       154
weighted avg       0.48      0.64      0.53       154

Accuracy_score 0.6428571428571429


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0

In [12]:
from xgboost import XGBClassifier
XGB = XGBClassifier()
XGB.fit(X_train, y_train)
y_pred = XGB.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
cr=(classification_report(y_test, y_pred))
acc = accuracy_score(y_pred,y_test)
print('Confusion matrix:')
print(cm)
print('Classification_report:')
print(cr)
print('Accuracy_score',acc)

Confusion matrix:
[[88  7  6  0]
 [16  2  4  1]
 [16  1  2  0]
 [ 9  0  2  0]]
Classification_report:
              precision    recall  f1-score   support

           0       0.68      0.87      0.77       101
           1       0.20      0.09      0.12        23
           2       0.14      0.11      0.12        19
           3       0.00      0.00      0.00        11

    accuracy                           0.60       154
   macro avg       0.26      0.27      0.25       154
weighted avg       0.49      0.60      0.53       154

Accuracy_score 0.5974025974025974


In [13]:
test=pd.read_csv('test.csv')
# print(test)
t_df=pd.DataFrame(test)
# print(t_df)
t_df.drop('Loan_ID',axis=1,inplace=True)
# print(t_df)
to_numeric = {'Male': 1, 'Female': 2,
'Yes': 1, 'No': 2,
'Graduate': 1, 'Not Graduate': 2,
'Urban': 3, 'Semiurban': 2,'Rural': 1,
'Y': 1, 'N': 0,
'3+': 3}
t_df = t_df.applymap(lambda label: to_numeric.get(label) if label in to_numeric else label)
Dependents_ = pd.to_numeric(t_df.Dependents)
t_df.drop(['Dependents'], axis = 1, inplace = True)
t_df = pd.concat([t_df, Dependents_], axis = 1)
print(t_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             356 non-null    float64
 1   Married            367 non-null    int64  
 2   Education          367 non-null    int64  
 3   Self_Employed      344 non-null    float64
 4   ApplicantIncome    367 non-null    int64  
 5   CoapplicantIncome  367 non-null    int64  
 6   LoanAmount         362 non-null    float64
 7   Loan_Amount_Term   361 non-null    float64
 8   Credit_History     338 non-null    float64
 9   Property_Area      367 non-null    int64  
 10  Dependents         357 non-null    float64
dtypes: float64(6), int64(5)
memory usage: 31.7 KB
None


In [14]:
t_df.isnull().sum().sort_values(ascending=False)

Credit_History       29
Self_Employed        23
Gender               11
Dependents           10
Loan_Amount_Term      6
LoanAmount            5
Married               0
Education             0
ApplicantIncome       0
CoapplicantIncome     0
Property_Area         0
dtype: int64

In [15]:
null_cols = ['Credit_History', 'Self_Employed','Gender','Dependents', 'Loan_Amount_Term', 'LoanAmount']

for col in null_cols:
    t_df[col] = t_df[col].fillna(
    t_df[col].dropna().mode().values[0] )   

t_df.isnull().sum().sort_values(ascending=False)

Gender               0
Married              0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Dependents           0
dtype: int64

In [16]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X, y)
X_test = t_df.iloc[:,:].values
y_pred=classifier.predict(X_test)
print(y_pred)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
